<a href="https://colab.research.google.com/github/ad71/ragbot/blob/master/langgraph_chat_agent_executor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -qU langchain langchain_openai tavily-python langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.2/121.2 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 771.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = userdata.get('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')

In [22]:
# MODIFICATION FOR DYNAMIC RETURN

from langchain_core.pydantic_v1 import BaseModel, Field


class SearchTool(BaseModel):
    query: str = Field(description='query to look up online')
    return_direct: bool = Field(
        description='Whether or not the result of this should be returned directly to the user without you seeing what it is',
        default=False
    )

from langchain_community.tools.tavily_search import TavilySearchResults
tools = [TavilySearchResults(max_results=1, args_schema=SearchTool)]

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
tools = [TavilySearchResults(max_results=1)]

In [23]:
from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

In [24]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=0, streaming=True) # stream tokens idk why

In [25]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind_functions(functions)

In [26]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage


class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add] # list of messages

In [27]:
# define the nodes
from langgraph.prebuilt import ToolInvocation
from langchain_core.messages import FunctionMessage
import json

def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]

    if 'function_call' not in last_message.additional_kwargs:
        return 'end'
    else:
        return 'continue'

In [28]:
# MODIFICATION:
# return direct if required
def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]

    if 'function_call' not in last_message.additional_kwargs:
        return 'end'

    else:
        arguments = json.loads(last_message.additional_kwargs['function_call']['arguments'])

        if arguments.get('return_direct', False):
            return 'final'
        else:
            return 'continue'

In [29]:
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)

    return {'messages': [response]} # we return a list because this will get added to the existing list

In [30]:
# standard call_tool
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]

    action = ToolInvocation(
        tool=last_message.additional_kwargs['function_call']['name'],
        tool_input=json.loads(last_message.additional_kwargs['function_call']['arguments'])
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {'messages': [function_message]}

In [16]:
# MODIFICATION: human in the loop
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]

    action = ToolInvocation(
        tool=last_message.additional_kwargs['function_call']['name'],
        tool_input=json.loads(last_message.additional_kwargs['function_call']['arguments'])
    )

    response = input(prompt=f'[y/n] continue with {action}?')

    if response == 'n':
        raise ValueError

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {'messages': [function_message]}

In [31]:
# MODIFICATION: return direct if required
def call_tool(state):
    messages = state['messages']
    last_message = messages[-1]

    tool_name = last_message.additional_kwargs['function_call']['name']
    arguments = json.loads(last_message.additional_kwargs['function_call']['arguments'])

    # delete parameter because if we've reached the call_tool stage, the tool doesn't need to know
    if tool_name == 'tavily_search_results_json':
        if 'return_direct' in arguments:
            del arguments['return_direct']

    action = ToolInvocation(
        tool=tool_name,
        tool_input=arguments
    )

    response = tool_executor.invoke(action)
    function_message = FunctionMessage(content=str(response), name=action.tool)
    return {'messages': [function_message]}

In [17]:
# define the graph

from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node('agent', call_model)
workflow.add_node('action', call_tool)

workflow.set_entry_point('agent')

workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'action',
        'end': END
    }
)

workflow.add_edge('action', 'agent')
app = workflow.compile()

In [33]:
# MODIFICATION: return direct graph
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node('agent', call_model)
workflow.add_node('action', call_tool)
workflow.add_node('final', call_tool)

workflow.set_entry_point('agent')

workflow.add_conditional_edges(
    'agent',
    should_continue,
    {
        'continue': 'action', # if `tools`, then we call the tool node
        'final': 'final', # final call
        'end': END # otherwise we finish
    }
)

workflow.add_edge('action', 'agent')
workflow.add_edge('final', END)

app = workflow.compile()

In [18]:
from langchain_core.messages import HumanMessage

inputs = {'messages': [HumanMessage(content='what is the weather in SF')]}
app.invoke(inputs)

[y/n] continue with tool='tavily_search_results_json' tool_input={'query': 'weather in San Francisco'}?y


{'messages': [HumanMessage(content='what is the weather in SF'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-e09b0ade-3d52-45a4-8b14-6f15cb206a35-0'),
  FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1715955418, \'localtime\': \'2024-05-17 7:16\'}, \'current\': {\'last_updated_epoch\': 1715955300, \'last_updated\': \'2024-05-17 07:15\', \'temp_c\': 12.2, \'temp_f\': 54.0, \'is_day\': 1, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/122.png\', \'code\': 1009}, \'wind_mph\': 5.6, \'wind_kph\': 9.0, \'wind_degree\': 260, \'wind_dir\': \'

In [35]:
# for human in the loop, use streaming
inputs = {'messages': [HumanMessage(content='what is the weather in sf? return this result directly by setting return_direct = True')]}
for output in app.stream(inputs):
    for key, value in output.items():
        print(f'Output from node {key}: ')
        print('----')
        print(value)

    print('\n----\n')

Output from node agent: 
----
{'messages': [AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco","return_direct":true}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'}, id='run-d295b4b4-6e7b-4b53-9769-b245eb08e0c0-0')]}

----

Output from node final: 
----
{'messages': [FunctionMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1715957314, \'localtime\': \'2024-05-17 7:48\'}, \'current\': {\'last_updated_epoch\': 1715957100, \'last_updated\': \'2024-05-17 07:45\', \'temp_c\': 12.2, \'temp_f\': 54.0, \'is_day\': 1, \'condition\': {\'text\': \'Overcast\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/122.png\', \'code\': 1009}, \'wind_mph\': 5.6, \'wind_kp